## Demo for Cath

In [1]:
from pathlib import Path
import sys

module_path = Path.cwd().parent.as_posix()
if module_path not in sys.path:
    sys.path.append(module_path)

In [35]:
from datetime import datetime as dt
import random
from time import sleep

### Custom functions for simulating trials

These are in experiment.py

In [38]:
def set_data_dir():
    DATA_DIR = Path.cwd().parent / "data"  # running code from notebooks
    if not DATA_DIR.exists():
        DATA_DIR = (
            Path.cwd() / "data"
        )  # running code from src - TODO: check if this works
    print(f"Data directory is: {DATA_DIR}")
    return DATA_DIR

In [39]:
DATA_DIR = set_data_dir()

Data directory is: /Users/mjboothaus/code/github/databooth/cath-henshall/data


### Data files

The data files are in the data folder. The data files are named according to the following convention:
- there is a master log file which contains a complete record of all trials (where they are "successful" or not)
- Latency data - for each TODO - clarify this

In [23]:
# Trial parameters

# TODO: Maybe specify these in a config file?
# QUESTION: Will these parameters change for trials (e.g. do they need to be calibrated in the initial experiments)?

ACTIVATION_TIMEOUT_SEC = 5
WAIT_AFTER_CORRECT_RESPONSE_SEC = 3
FEED_CONSUMPTION_TIMEOUT_SEC = 15
TRIAL_TIMEOUT_SEC = 60
START_TONE_SEC = 1

In [11]:
trial_parameters_names = [
    "ACTIVATION_TIMEOUT_SEC",
    "WAIT_AFTER_CORRECT_RESPONSE_SEC",
    "FEED_CONSUMPTION_TIMEOUT_SEC",
    "TRIAL_TIMEOUT_SEC",
]

In [34]:
from src.experiment import (
    log_trial_parameters,
    log_event,
    play_beep,
    play_correct_response_tone,
    dispense_feed,
    wait_for_period_of_time,
    wait_for_start_button_press,
)

In [22]:
# Experiment trial


def run_trials(subject_id, n_trials=10):
    log_trial_parameters()

    # Event 0: Horse enters test chute - how does the experimenter know when it is ok to start the trial?

    log_event(f"Subject {subject_id} entering test chute", dt.now())
    wait_for_start_button_press()

    # Event 1: Start tone - 1 sec duration

    log_event(f"Playing buzzer for {START_TONE_SEC} seconds", dt.now())
    play_beep(START_TONE_SEC)
    log_event("Start buzzer finished", dt.now())

    # Event 2: Potential activation of touch sensor (nose press on panel)

    start_sensor_period = dt.now()
    touch_sensor_activated = False

    # Allow for Phases 1, 2 and 3 (note that Phases 1 and 3 are the same)
    # Also allow for a specific number of trials

    while (
        dt.now() - start_sensor_period
    ).seconds < ACTIVATION_TIMEOUT_SEC:  # Simulate the horse either activating the touch sensor or not
        sleep(0.9)
        if random.random() < 0.2:  # simulate touch sensor activation
            touch_sensor_activated = True
            touch_time = dt.now()
            touch_latency = (touch_time - start_sensor_period).total_seconds()
            log_event(
                f"Touch sensor activated after {touch_latency} seconds", touch_time
            )
            # reset_touch_sensor()
            play_correct_response_tone()
            wait_for_period_of_time(
                "Wait after correct response", WAIT_AFTER_CORRECT_RESPONSE_SEC
            )
            dispense_feed()
            wait_for_period_of_time(
                "Waiting after feed dispensed", FEED_CONSUMPTION_TIMEOUT_SEC
            )
            break
    if not touch_sensor_activated:
        log_event(
            f"Touch sensor not activated after {ACTIVATION_TIMEOUT_SEC} seconds",
            dt.now(),
        )

    # Event 3: Correct response tone - 0.5 sec duration

    log_event("Trial ends", dt.now())

Logged - TRIAL PARAMETERS: {'ACTIVATION_TIMEOUT_SEC': 5, 'WAIT_AFTER_CORRECT_RESPONSE_SEC': 3, 'FEED_CONSUMPTION_TIMEOUT_SEC': 15, 'TRIAL_TIMEOUT_SEC': 60}

Logged - 2023-06-26 14:08:58.457977: Horse enters test chute

Logged - 2023-06-26 14:09:08.760576: Trial started

Logged - 2023-06-26 14:09:08.761892: Playing buzzer for 1 seconds

Logged - 2023-06-26 14:09:09.856201: Start buzzer finished

Logged - 2023-06-26 14:09:12.565650: Touch sensor activated after 2.708457 seconds

Logged - 2023-06-26 14:09:12.566653: Start playing correct response tone

Logged - 2023-06-26 14:09:13.634323: Finished playing correct response tone

Logged - 2023-06-26 14:09:13.636638: Waiting after correct response finished for 3 seconds

Logged - 2023-06-26 14:09:13.637682: Wait after correct response: Waiting for 3 seconds

Logged - 2023-06-26 14:09:16.642615: Wait after correct response: Finished waiting for 3 seconds

Logged - 2023-06-26 14:09:16.643442: Dispensing feed

Logged - 2023-06-26 14:09:16.64396